# Submodule 1: Preprocessing and Quality Control

<img src="images/LessonPlan.jpg" alt="Drawing" width=1000 />

## Overview & Purpose
## ChIP-seq, CUT&RUN, and CUT&Tag
This short tutorial demonstrates the initial processing steps for ChIP-seq, CUT&RUN, and CUT&Tag analysis. In this first module, we focus on generating quality reports of the raw sequences, adapter trimming, mapping, and removal of PCR duplicates. 

<img src="images/all3methods.jpg" alt="Drawing" width=500 />

To demonstrate the process, this tutorial will analyze published datasets studying H3K27ac occupancy after aux-mediated degradation of BAF (ChIP-seq), as well as BAF (CUT&RUN) and RNA Polymerase II Ser5ph (CUT&Tag) occupancy after inhibition of transcription elongation using Flavopiridol. As such, this module covers the processing of the data from three distinct but similar methods using downsampled data to improve runtime speed. The original data can be found in the GEO repository with the following accessions: ChIP-seq - GSE173550; CUT&RUN & CUT&Tag - GSE224292. These data were published in the following articles:

Weber CM, et al. mSWI/SNF promotes Polycomb repression both directly and through genome-wide redistribution. Nat Struct Mol Biol. 2021  PMID: [34117481](https://pubmed.ncbi.nlm.nih.gov/34117481/)

Brahma S, Henikoff S. The BAF chromatin remodeler synergizes with RNA polymerase II and transcription factors to evict nucleosomes. Nat Genet. 2024 PMID: [38049663](https://pubmed.ncbi.nlm.nih.gov/38049663/)

Note that to allow faster processing we have limited the reads to that of a single chromosome (chr4).  

While ChIP-seq, CUT&RUN, and CUT&Tag are all used to identify chromatin binding sites genome-wide, they differ in implementation which can impact the analysis.
<img src="images/methodcompare.png" alt="Drawing" style="width: 800px;"/>
Image credit: Kaya-Okur et al., 2020. Efficient low-cost chromatin profiling with CUT&Tag. Nature Protocols.

## ChIP-seq
This method uses random fragmentation followed by immunoprecipitation.

<img src="images/chipseq.gif" alt="Drawing" style="width: 500px;" align="center"/>

## CUT&RUN
This method uses random pA-MNase to target the fragmentation to the site where your protein is bound.

<img src="images/CUTRUN.gif" alt="Drawing" style="width: 500px;" align="center"/>

## CUT&Tag
This method uses random pA-Tn5 to directly insert sequencing adapters next to where your protein is bound.

<img src="images/CUTTag.gif" alt="Drawing" style="width: 500px;" align="center"/>


### Ways to use this module
Throughout this module, we have color-coded commands according to ChIP-seq, CUT&RUN, and CUT&Tag. Therefore this module can be used to learn about the processing of each method individually, to compare each method to the others, or you can follow the colored commands to only process one type, either ChIP-seq, CUT&RUN, or CUT&Tag.
Commands for each method will be designated by an individual logo before the command, just like the following examples

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [ ]:
#run this cell for ChIP-seq
print("Code for ChIP-seq will be placed after the above image. Run these cells if performing ChIP-seq analysis.")

<img src="images/CUT&RUNLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [ ]:
#run this cell for CUT&RUN
print("Code for CUT&RUN will be placed after the above image. Run these cells if performing CUT&RUN analysis.")

<img src="images/CUT&TagLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [ ]:
#run this cell for CUT&Tag
print("Code for CUT&Tag will be placed after the above image. Run these cells if performing CUT&Tag analysis.")

<div class="alert alert-block alert-success" style="font-size:100%">
<span style="color:black"> By following the colors/images, you can run one, two, or all three types of analyses.</span>
</div>

### Required Files
In this stage of the module, you will use the fastq files that have been prepared. You can also use this module on your own data or any published ChIP-seq, CUT&RUN, or CUT&Tag dataset. 

<div class="alert-info" style="font-size:200%">
STEP 1: Set Up Environment
</div>

Initial items to configure your Cloud environment. In this step we will use conda to install the following packages:

Quality Reporting:
[fastqc](https://anaconda.org/bioconda/fastqc), [multiqc](https://anaconda.org/bioconda/multiqc)

Read Trimming: 
[trimmomatic](https://anaconda.org/bioconda/trimmomatic)

Mapping:
[bowtie2](https://anaconda.org/bioconda/bowtie2)

In [ ]:
#First let's install mamba to configure our environment
! curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"
! bash Miniforge3-$(uname)-$(uname -m).sh -u -b -p $HOME/mambaforge
print("done")

In [ ]:
#now let's install several required packages
!mamba install -c bioconda fastqc bowtie2 multiqc trimmomatic -y
!pip install jupyterquiz==2.0.7 jupytercards
#!pip install --user igv-notebook
print("done")

<div class="alert alert-block alert-warning" style="font-size:100%">
<span style="color:black"> The next cell will restart the kernel, so you will see the following message once you run the cell. When this image appears, click okay, and then keep going onto the next cell.</span>
</div>

<img src="./images/restartkernel.jpg" alt="Drawing" style="width: 500px;" align="left"/>

In [ ]:
#Restarts the kernel to implement changes 
import os
os._exit(00)

<div class="alert alert-block alert-warning" style="font-size:100%">
<span style="color:black"> After the kernel restarts continue from here.</span>
</div>

In [ ]:
#Now let's import packages that we installed
numthreads=!lscpu | grep '^CPU(s)'| awk '{print $2-1}'
numthreadsint = int(numthreads[0])
import sys
from jupyterquiz import display_quiz
from IPython.display import IFrame
#from IPython.display import display
from jupytercards import display_flashcards
import pandas as pd
#import modules for matching-type quiz
%cd questions
from quiz_module import run_quiz
%cd ../
import json
import ipywidgets as widgets
from IPython.display import display
import random
print("done")

In [ ]:
%cd ~/SageMaker/SandboxChromatinOccupancy
#show which folder you are working in. 
!pwd

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question: Match the method to how it implements chromatin fragmentation.
</div>

In [ ]:
#Run for the quiz
run_quiz("./questions/methodvsfragmentation.json", instant_feedback=True, shuffle_questions=True, shuffle_answers=True)

In [ ]:
# These commands move into our Tutorial 1 directory and create our subdirectory structure.
!mkdir -p Tutorial1/
%cd Tutorial1/
!mkdir -p QC
!mkdir -p Trimmed
!mkdir -p Mapped

In [ ]:
#show which folder you are working in. It should now say Tutorial1 at the end.
!pwd
#list the files/folders in the current directory. You should see QC, Trimmed, and Mapped.
!ls

In [ ]:
#Let's copy and extract our tutorial files
!wget https://chromatinoccupancytutorial.s3.us-east-2.amazonaws.com/Submodule1.zip
!unzip Submodule1.zip

In [ ]:
#list the files in ChIPseqFiles which we just downloaded.
!ls ChIPseqFiles

<div class="alert-info" style="font-size:200%">
STEP 2: QC
</div>

Sequences are typically provided as files in fastq format. This format includes four lines per sequence.


In [ ]:
display_flashcards('../flashcards/FastqFlashCard.json')

### Click on the above image to see what each line represents.
Next, let's take a look at the sequence quality of the raw reads using fastqc:

In [ ]:
# This command runs fastqc on each fastq.gz file inside our InputFiles directory and stores the ouput reports in our QC directory.
!fastqc -t $numthreadsint -q -o QC ChIPseqFiles/*fastq.gz

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 100px;" align="left"/>

In [ ]:
#Run this cell if following the ChIP-seq tutorial.
# We can display the resulting fastqc results.
IFrame(src='Tutorial1/QC/H3K27ac_ChIPseq_noaux_fastqc.html', width=1080, height=800)

<div class="alert-success" style="font-size:100%">
After you've browsed the above report, try to visualize the quality of the other sample(s). Use the cell below to adapt the commands to visualize the bafi sample.
</div>

<div class="alert-warning" style="font-size:150%; color:black">
Try it yourself! Within the following block, try to type out the command for fastqc on the other sample
</div>

In [ ]:
#Type the code to visualize the fastqc report on the other sample: Tutorial1/QC/H3K72ac_ChIPseq_aux_fastqc.html



<div class="alert-info" style="font-size:200%">
Trimming
</div>
<img src="images/trimming.jpg" alt="Drawing" style="width: 250px;"/>
Now that we've viewed the quality let's trim the sequences to remove poor-quality bases and any adapter contamination. We'll use a package called Trimmomatic.

## Let's use trimmomatic to prepare the sequences before mapping.

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 100px;" align="left"/>

In [ ]:
#Run this cell if following the ChIP-seq tutorial.
# This will trim off N's as well as nextera adapters present in the ChIPseq library preparation. Placing the trimmed reads in our Trimmed folder.
!trimmomatic SE -threads $numthreadsint ./ChIPseqFiles/H3K27ac_ChIPseq_noaux.fastq.gz Trimmed/H3K27ac_ChIPseq_noaux_trimmed.fastq.gz ILLUMINACLIP:RefGenome/ChIPseqAdapters.fa:2:30:10 LEADING:3 TRAILING:3
#This next command is the same, except we'll do it for the bafi sample.
!trimmomatic SE -threads $numthreadsint ./ChIPseqFiles/H3K27ac_ChIPseq_aux.fastq.gz Trimmed/H3K27ac_ChIPseq_aux_trimmed.fastq.gz ILLUMINACLIP:RefGenome/ChIPseqAdapters.fa:2:30:10 LEADING:3 TRAILING:3

print("done")

<div class="alert-info" style="font-size:200%">
STEP 3: Mapping
</div>
Our fastq files include sequences and quality scores for each base, but we want to figure out which genomic location these sequences came from. To do this we will map each sequence to a reference genome using bowtie2. 

<img src="images/ReferenceGenome.png" alt="Drawing" style="width: 400px;"/>

Genome Index

Finding a matching sequence in billions of base pairs of sequence is a difficult task. Consider how much time it would take to match 20 million reads of sequence! 

To facilitate this process, alignment tools use a genome index. Think of this like a book index. The genome index is a fast way for the tool to look up specific "seeds" of sequence (seeds are a small portion of the sequence that can match multiple locations). Once it has the locations of the "seed" portion, it can then narrow down to the exact location based on the entire sequence of the read.

<img src="images/indexsearch.jpg" alt="Drawing" style="width: 300px;"/>

There are several different alignment tools, and each uses a different way of creating the index. We will use a tool called bowtie2, so we'll need indexes created by bowtie2. Because this is a long but easy process, I'm going to give you the command, but you won't be expected to run it. 

Mapping reads requires a reference genome. Due to time and memory considerations, in this tutorial we  prepared that file for you and will only map to chr4. However, in a full analysis, we would map to the entire genome. To do so you would need a fasta file corresponding to the reference genome (e.g. [mm39](https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M36/GRCm39.primary_assembly.genome.fa.gz)) from which you'd create an index of the genome using bowtie2-build. This can be done with the command: 

bowtie2-build reference_genome_file.fa outputprefix.

As mentioned, we've gone ahead and created the index for you, and, earlier, you extracted them into the RefGenome directory. These index files end in the bt2 extension. 

In [ ]:
!ls RefGenome/*bt2

<div class="alert-info" style="font-size:100%">
These index files were created from our fasta file:
 </div>

In [ ]:
# Notes: The -x option specifies the prefix of the index. -U specifies our fastq file (use -1 and -2 if you have paired-end reads). -S specifies our output file in sam format.
!bowtie2 -p $numthreadsint -x RefGenome/mm39_chr4 -U Trimmed/H3K27ac_ChIPseq_noaux_trimmed.fastq.gz -S Mapped/H3K27ac_ChIPseq_noaux.sam
#This runs bowtie2 again, for the other sample.
!bowtie2 -p $numthreadsint -x RefGenome/mm39_chr4 -U Trimmed/H3K27ac_ChIPseq_aux_trimmed.fastq.gz -S Mapped/H3K27ac_ChIPseq_aux.sam

# Bowtie2 outputs a file in [sam format](https://samtools.github.io/hts-specs/SAMv1.pdf), which contains the original sequence, quality scores, and the genomic coordinates matching each read. 

<img src="images/samformat.jpg" alt="Drawing" style="width: 800px;"/>

Each read is also given a SAM flag (second column). This is a numerical code corresponding to various features of the mapping.

A flag of 0 means the read was single-end and mapped to the plus strand. A flag 16 means the read was single-end and mapped to the minus strand. A flag of 4 means the read was unmapped. These are the most common values you will see with single-end data. 

You can also look up what different SAM flag numerical codes mean: https://broadinstitute.github.io/picard/explain-flags.html

<img src="images/explainsam.jpg" alt="Drawing" style="width: 400px;"/>

### While we obtained high alignment rate, let's discuss a few features that can impact alignment. 

Remember that some of the genome is repetitive and so some of our reads may match multiple locations. Bowtie2 reports the location of each read, as well as a Map Quality score for each. This quality score is impacted by several factors, including how many loci it could match as well as the base quality (confidence in the sequence) we received. 

If you expect issues with repetitive regions (for example if your protein is abundant near pericentromeres or transposons) then using longer reads can help. For most purposes, we can get by with fairly short reads. 

<img src="images/repetitive.gif" alt="Drawing" style="width: 400px;"/>

<div class="alert alert-block alert-success" style="font-size:120%">
<span style="color:black">Congrats! You have successfully preprocessed and mapped the samples. In the next tutorial, we will continue with the processing by filtering for quality, visualizing the signal, identifying peaks, and performing differential analysis.</span>
</div>


# Continue on to [Tutorial2](../Tutorial2.ipynb)